In [ ]:
import course;course.header()

# Pandas level 1
Data wrangling 101

I'd like to say Pandas is numpy on steriods but it is actually much more.

Pandas is the data science solution for Python and it build ontop of the powerful numpy module.
However, Pandas offers elements that are much more intuitive or go beyond what numpy has ever provided.
Nevertheless, numpy is more performant in some cases (by a lot, yet remember when to optimize!) 

The perfect is the dead of the good.
 -- M. Gunner

Pandas was create [Wes McKinney](https://wesmckinney.com/pages/about.html) in the early 2008 at AQR capital management and I can recommend "Python for Data Analysis" from Wes, which was published via O'Reilly and "Pandas for Everyone" by Daniel Y. Chen. The following Pandas chapters are inspired by the books.

Pandas offers the two basic data structures
* Series
* Dataframes


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
c = pd.Series(
    np.random.randn(4), 
    index=['r1', 'r2', 'r3', 'r4']
)
c

Selecting from Series works like a dict :)

In [ ]:
c['r2']

In [ ]:
mask = c > 0
mask

In [ ]:
c[mask]

Masks can be additive!

In [ ]:
mask2 = c < 1
c[mask & mask2]

In [ ]:
c * 10

In [ ]:
np.exp(c)

Remember to use numpy functions as much as possible so data remains on the "C side". More below!

Operations conserve index!

Series are like ordered Dicts!

In [ ]:
'r1' in c

np.nan is the missing value indicator

In [ ]:
d = pd.Series({'r1': np.nan, 'r2': 0.2, 'r3': 0.2, 'r4': 0.4})

In [ ]:
d

In [ ]:
d.isna()

In [ ]:
# inverting with ~!
~d.isna()

In [ ]:
d.notnull()

## indices are aligned automatically!

In [ ]:
c

In [ ]:
d = pd.Series(
    np.random.randn(4), 
    index=['r2', 'r3', 'r4', 'r5']
)
d

In [ ]:
c + d

## Renaming index

In [ ]:
d.index = ['r1', 'r2', 'r3', 'r4']

In [ ]:
c + d

Naming things will help you to get your data organised better. Explicit is better than implicit! And remember to choose your names variable wisely - you will code read often than you write.  

In [ ]:
d.index.name = "variable"
d.name = "probability"
d

In [ ]:
d.reset_index()

Turns it into a DataFrame as the index is now a series!

In [ ]:
type(d.reset_index())

# Data frames 
Data frames are the pandas 2d data containers (if there is only one index dimension). 
In principle data frames are a list of Series, whereas each row is a series. 

In [ ]:
df = pd.DataFrame(
    [
        c, 
        d, # this one we named :)
        pd.Series(np.random.randn(4), index=['r2', 'r3', 'r4', 'r5'])
    ]
)
df

In [ ]:
# accessing a value
df.loc['probability', 'r2']

Note: How pandas aligns your data automatically.

If you want each series to be treated as column, just transpose

DataFrames can be constructed in many different ways, see docu for more details
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html?highlight=dataframe#pandas.DataFrame

In [ ]:
df = df.T
df

Renaming columns in a data frame

In [ ]:
df.columns = ['p1', 'p2', 'p3']
df

Dataframes can equally be named, for your sanity, name them :)

In [ ]:
df.columns.name = "probability"
df.index.name = "variable"
df

Now that you feel happy in the pandas world, some modules/functions require numpy arrays, how do you convert them ?

In [ ]:
np_df = df.values
np_df

In [ ]:
type(np_df)

If you need to work "longer" on the numpy side, I suggest to transform the pandas dataframe to a numpy recarray, as names are preserved; 

In [ ]:
# np_df = df.values # 
np_df = df.to_records()
np_df

In [ ]:
np_df['variable']

In [ ]:
np_df[0]

In [ ]:
np_df[0][2]

## C-side and Python side 

**Note**:
Regular Python floats live in the Python world - Numpy and Pandas live in the "C world", hence their fast vectorized operations. If you can avoid it, don't cast between the worlds! 

In [ ]:
long_series = pd.Series(
    np.random.randn(1000000), 
)

In [ ]:
%%timeit -n 1
a = long_series.to_list()  # to python list!
print(f"a is a {type(a)} now!")
pd.Series(a)

In [ ]:
%%timeit -n 1
a = long_series.to_numpy()
print(f"a is a {type(a)} now!")
pd.Series(a)

# Operations between DataFrame and Series

In [ ]:
df_small = pd.DataFrame([c, d])
df_small

In [ ]:
c

In [ ]:
df_small - c

Next time you want to normalize each row of a data frame, one can define the correction factors as a series and just e.g. subtract it. 

In [ ]:
# renaming columns

In [ ]:
df

In [ ]:
df.rename(columns={'p1':'VLC'}, inplace=True)

In [ ]:
df

In [ ]:
# subselecting a set of columns! 
df[["VLC", 'p2']]

**Note:**
This only creates a view of the data! 

# Pandas IO
Pandas comes with a wide array of input output modules see
https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html

**NOTE:** reading xlsx is _much_ slower than csv

Your request: Scraping websites! 

Today with Pandas scraping wikipedia. In particular the oldest universities!

Alternatively beautiful soup https://www.crummy.com/software/BeautifulSoup/bs4/doc/ or Scrapy https://scrapy.org/

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_oldest_universities_in_continuous_operation"

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
dfs = pd.read_html(url) # do you get SSL: CERTIFICATE_VERIFY_FAILED ?

In [ ]:
len(dfs)

In [ ]:
dfs[1]

In [ ]:
udf = dfs[1]

In [ ]:
udf.columns

In [ ]:
udf.columns = [ e[0] for e in udf.columns ]

In [ ]:
udf.head(2)

In [ ]:
udf.columns = ['Year', 'University', 'H-Location', 'G-Location', 'Notes' ]

In [ ]:
udf.head()

## Gather some basic information around the dataframe

In [ ]:
udf.describe()

In [ ]:
udf.info()

We need to clean-up the Year

Accessing the str properties!

In [ ]:
udf['Year'].str.match(r'^(?P<year>[0-9]{4})')

In [ ]:
udf['year'] = udf.Year.str.extract(r'(?P<year>[0-9]{4})')

In [ ]:
udf.head()

In [ ]:
udf.shape
# (rows, columns)

One cannot visualize all columns straight away in jupyter :( However redefining some options helps!

In [ ]:
pd.set_option("max_columns", 2000)

# Sorting

In [ ]:
udf.head()

In [ ]:
udf.sort_values(['year'])

Sort_values has kwargs like ascending = True|False and values are defined by a list, ie sort first by, then by ...

In [ ]:
udf.sort_values(['year', 'G-Location'])

Let split the G-location into city and country!

In [ ]:
tmp_df = udf['G-Location'].str.split(",")
display(tmp_df.head())   # not quite what we want .. we want two columns!

How to get two columns?

In [ ]:
tmp_df = udf['G-Location'].str.split(",", expand=True)
tmp_df.columns = ['G-City', 'G-Country']

In [ ]:
tmp_df

In [ ]:
udf = udf.join(tmp_df)
# there are many options to join frames 

In [ ]:
udf.head()

# Deleting things

In [ ]:
udf.head()

In [ ]:
udf.drop(1)

In [ ]:
udf.head(3)

In [ ]:
udf.drop(columns=['G-Location', 'Year'])

Dataframe or series are not automatically "adjusted" except you use `inpace=True`

In [ ]:
udf

In [ ]:
udf.drop(columns=['G-Location', 'Year'], inplace=True)

In [ ]:
udf


# slicing and dicing

In [ ]:
udf[:3] # df[:'r3'] works as well

In [ ]:
# selecting one column!
udf['G-Country']

In [ ]:
# selecting one row
udf.loc[1]

In [ ]:
udf.info()

In [ ]:
# mask also work on df!
mask = udf['year'] < 1400
mask.head(10)

In [ ]:
udf.year = udf.year.astype(int)

In [ ]:
_udf = udf.convert_dtypes()

In [ ]:
_udf.info()

In [ ]:
# mask also work on df!
mask = udf.year < 1400
mask.head(10)

In [ ]:
udf[mask]

In [ ]:
udf[udf['year'] < 1300] # reduces the data frame, again note! that is just a view, not a copy!

In [ ]:
udf[udf['year'] < 1300].loc[1]

In [ ]:
udf[udf['year'] > 1300].loc[1]

In [ ]:
udf[udf['year'] > 1300].head(3)

In [ ]:
udf[udf['year'] > 1300].iloc[1]

## more natural query - or isn't it?

In [ ]:
udf.query("year > 1300").head(5)

In [ ]:
udf.query("1349 > year > 1320")

In [ ]:
# Using local variables in queries
upper_limit = 1400
udf.query("@upper_limit > year > 1320")

## Find the maximum for a given series or dataframe

In [ ]:
udf['year'].idxmax()

## Unique values and their count

In [ ]:
udf['G-Country'].unique()

In [ ]:
udf['G-Country'].nunique()

In [ ]:
udf['G-Country'].value_counts()

In [ ]:
_udf = udf.set_index('University')

In [ ]:
_udf.sample(5)

In [ ]:
_udf.loc['University of Florence', ['Notes', 'year']]

In [ ]:
_udf.loc['University of Florence', :]

## Done with Basics!
Take a look at the cheat sheet for a summary
https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

# Hierarchical indexing

In [ ]:
s = pd.Series(
        np.random.randn(5), 
        index = [
            ['p1','p1','p2','p2','p3'],
            ['a','b','a','d','a']
        ]
)
s

In [ ]:
s.index

In [ ]:
s.index.names = ['probability', 'type']

In [ ]:
s

In [ ]:
s['p1']

In [ ]:
s[:, 'a'] # lower level 

In [ ]:
s2 = s.unstack()
print(type(s2))
s2

In [ ]:
s3 = s2.stack()
print(type(s3))
s3

## Multindex with Dataframes

In [ ]:
df = pd.DataFrame(
    [
        c, 
        c * 20, 
        d,
        np.exp(d),
        pd.Series(np.random.randn(4), index=['r2', 'r3', 'r4', 'r5'])
    ],
    index = [
        ['p1','p1','p2','p2','p3'],
        ['a','b','a','d','a']
    ]
)
df.index.names = ['probability', 'type']
df

In [ ]:
df = df.fillna(0)
df

**Note**:
You can create multi indeces from a regular dataframe!

In [ ]:
df2 = df.reset_index()

In [ ]:
df2

In [ ]:
df2.set_index(['probability', 'type'])

In [ ]:
df2 = df.swaplevel('probability', 'type')
df2

In [ ]:
df2.sort_index(axis=0, level=0)

## Natural slicing using `pandas.IndexSlice`  objects

In [ ]:
idx = pd.IndexSlice
df2.loc[idx[:, ["p1", "p2"]], :]

## long and wide formats

In [ ]:
df3 = df2.reset_index()
df3

In [ ]:
df4 = df3.melt(
    id_vars=['type','probability'],
    var_name='r_stage',
    value_name='score'
)
print(df4.shape)
df4.head()

In [ ]:
df5 = df4.pivot_table(index=['type', 'probability'], columns='r_stage', values="score")
df5 